In [24]:
import pandas as pd
import plotly.express as px

In [25]:
df = pd.read_csv('simulations_day_output.csv', sep=';', header=None)
df.columns = ['simulation_key', 'missing_plan_cnt_first_delay', 'missing_plan_cnt_planned_time', 'whole_process_delay']
df['controller_name'] = df.simulation_key.str.split('-', expand=True)[0]
df['break_strategy'] = df.simulation_key.str.split('-', expand=True)[1]
df['experiment_name'] = df.controller_name + '-' + df.break_strategy
df['simulation_index'] = df.simulation_key.str.split('-', expand=True)[2].astype(int)
df['produced_in_planned_time'] = 1440 - df.missing_plan_cnt_planned_time
df['whole_process_delay'] = df.whole_process_delay.str.replace(',', '.').astype(float)
df['process_delay_minutes'] = df.whole_process_delay / 60
df['process_delay_hours'] = df.process_delay_minutes / 60
df.head()

,simulation_key,missing_plan_cnt_first_delay,missing_plan_cnt_planned_time,whole_process_delay,controller_name,break_strategy,experiment_name,simulation_index,produced_in_planned_time,process_delay_minutes,process_delay_hours
0,naive-skip_break-0,0,0,0.0,naive,skip_break,naive-skip_break,0,1440,0.0,0.0
1,naive-reorganization-0,0,0,0.0,naive,reorganization,naive-reorganization,0,1440,0.0,0.0
2,async-skip_break-0,0,0,0.0,async,skip_break,async-skip_break,0,1440,0.0,0.0
3,async-reorganization-0,0,0,0.0,async,reorganization,async-reorganization,0,1440,0.0,0.0
4,naive-skip_break-1,0,0,0.0,naive,skip_break,naive-skip_break,1,1440,0.0,0.0


In [26]:
df.groupby('experiment_name')[['produced_in_planned_time', 'process_delay_minutes']].describe()

produced_in_planned_time                               \
                                        count     mean         std     min   
experiment_name                                                              
async-reorganization                     20.0  1439.85    0.489360  1438.0   
async-skip_break                         20.0  1439.65    0.670820  1438.0   
naive-reorganization                     20.0  1308.30  158.339110  1083.0   
naive-skip_break                         20.0  1308.25  158.295485  1083.0   

                                                       process_delay_minutes  \
                          25%     50%      75%     max                 count   
experiment_name                                                                
async-reorganization  1440.00  1440.0  1440.00  1440.0                  20.0   
async-skip_break      1439.75  1440.0  1440.00  1440.0                  20.0   
naive-reorganization  1124.00  1411.0  1417.75  1440.0                  20.0   
naive-skip_break      1124.00  1411.0  1417.75  1440.0                  20.0   

                                                                         \
                            mean         std  min        25%        50%   
experiment_name                                                           
async-reorganization    0.019333    0.026855  0.0   0.006667   0.010000   
async-skip_break        0.022000    0.026828  0.0   0.011667   0.013333   
naive-reorganization  155.589017  194.969325  0.0  20.033208  26.105000   
naive-skip_break      155.649417  194.973293  0.0  20.091042  26.202667   

                                              
                             75%         max  
experiment_name                               
async-reorganization    0.015000    0.083333  
async-skip_break        0.020000    0.083333  
naive-reorganization  373.689833  437.381000  
naive-skip_break      373.730375  437.430167

In [27]:
df_summaries = pd.DataFrame(index=df.experiment_name.unique(),
                            columns=['perc_complete_plans', 'perc_zero_delay_plans',
                                    'avg_missing_steps', 'avg_accumulated_delay',
                                    'missing_steps_90_percentile', 'missing_steps_95_percentile', 'missing_steps_99_percentile',
                                    'missing_steps_70_percentile', 'missing_steps_80_percentile'])

for group_name, df_group in df.groupby('experiment_name'):
    df_summaries.loc[group_name, 'perc_complete_plans'] = df_group[df_group.missing_plan_cnt_planned_time == 0].shape[0] / df_group.shape[0] * 100
    df_summaries.loc[group_name, 'perc_complete_plans'] = str(df_summaries.loc[group_name, 'perc_complete_plans']) + ' %'
    df_summaries.loc[group_name, 'perc_zero_delay_plans'] = df_group[df_group.whole_process_delay == 0].shape[0] / df_group.shape[0] * 100
    df_summaries.loc[group_name, 'perc_zero_delay_plans'] = str(df_summaries.loc[group_name, 'perc_zero_delay_plans']) + ' %'
    df_summaries.loc[group_name, 'avg_missing_steps'] = df_group.missing_plan_cnt_planned_time.mean()
    df_summaries.loc[group_name, 'avg_accumulated_delay'] = df_group.process_delay_minutes.mean()
    df_summaries.loc[group_name, 'missing_steps_90_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.9)
    df_summaries.loc[group_name, 'missing_steps_95_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.95)
    df_summaries.loc[group_name, 'missing_steps_99_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.99)
    df_summaries.loc[group_name, 'missing_steps_70_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.70)
    df_summaries.loc[group_name, 'missing_steps_80_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.80)

df_summaries

,perc_complete_plans,perc_zero_delay_plans,avg_missing_steps,avg_accumulated_delay,missing_steps_90_percentile,missing_steps_95_percentile,missing_steps_99_percentile,missing_steps_70_percentile,missing_steps_80_percentile
naive-skip_break,15.0 %,15.0 %,131.75,155.649,356,356.05,356.81,302.3,355
naive-reorganization,20.0 %,20.0 %,131.7,155.589,356,356.05,356.81,302.3,355
async-skip_break,75.0 %,20.0 %,0.35,0.022,1.1,2,2,0,1
async-reorganization,90.0 %,20.0 %,0.15,0.0193333,0.1,1.05,1.81,0,0


| Experiment | Plánovaný počet kusů | Procentuální počet splněných plánů | Procentuální počet plánů bez zpoždění | Průměrný počet vyprodukovaných kusů za plánované období 1-|
| --- | --- | --- | --- | --- |
| Naivní bez přeskládávání | 1416 | .608 | 0.161 | 1391.87 | 29.90

In [28]:
px.box(df, y='missing_plan_cnt_planned_time', color='experiment_name', title='Počet chybějících aut z plánu')

In [29]:
px.box(df, y='process_delay_minutes', color='experiment_name', title='Akumulovaný čas prostojů v minutách')

Maximalni delay uspesne vyrobene cele vyrobni rady

In [36]:
df[(df.missing_plan_cnt_planned_time == 0)].sort_values('whole_process_delay', ascending=False)

,simulation_key,missing_plan_cnt_first_delay,missing_plan_cnt_planned_time,whole_process_delay,controller_name,break_strategy,experiment_name,simulation_index,produced_in_planned_time,process_delay_minutes,process_delay_hours
38,async-skip_break-9,1382,0,5.0,async,skip_break,async-skip_break,9,1440,0.083333,0.001389
39,async-reorganization-9,1382,0,5.0,async,reorganization,async-reorganization,9,1440,0.083333,0.001389
19,async-reorganization-4,1380,0,4.8,async,reorganization,async-reorganization,4,1440,0.080000,0.001333
18,async-skip_break-4,1380,0,4.8,async,skip_break,async-skip_break,4,1440,0.080000,0.001333
63,async-reorganization-15,931,0,4.6,async,reorganization,async-reorganization,15,1440,0.076667,0.001278
31,async-reorganization-7,1379,0,1.2,async,reorganization,async-reorganization,7,1440,0.020000,0.000333
26,async-skip_break-6,1381,0,1.2,async,skip_break,async-skip_break,6,1440,0.020000,0.000333
30,async-skip_break-7,1379,0,1.2,async,skip_break,async-skip_break,7,1440,0.020000,0.000333
27,async-reorganization-6,1381,0,1.2,async,reorganization,async-reorganization,6,1440,0.020000,0.000333
46,async-skip_break-11,1375,0,1.2,async,skip_break,async-skip_break,11,1440,0.020000,0.000333


In [33]:
px.box(df[(df.missing_plan_cnt_planned_time == 0)], y='whole_process_delay', title='Délka prostoje v sekundách pro splněné plány')

In [32]:
px.box(df, y='missing_plan_cnt_first_delay', color='experiment_name', title='Počet chybějících plánovaných kroku při prvním prostoji')